# Super resolution notebook : training part
The slc data we want to use the super resolution on is stored in './data/train/slc_lr' and the denoised is stored in './data/train/denoised_lr'

## Parameters to modify
- device : 'cuda' to use gpu, 'cpu' otherwise
- model_name : network to use for super resolution, choose between :
    - 'SWINTRANSFORMER'
    - 'SRCNN'
    - 'PixelShuffle'
- SR_type : super resolution method, choose between :
    - 'n_n' for noisy input to noisy output
    - 'n_d' for noisy input to denoised output
    - 'd_d' for denoised input to denoised output

In [1]:
device = 'cuda' # cuda or cpu
model_name = 'SWINTRANSFORMER' # SRCNN, PixelShuffle, SWINTRANSFORMER 
SR_type = 'n_d' # n_n, n_d or d_d 

## Run all these cells to make super resolution with the parameters chosed above

In [2]:
import os
import yaml
import argparse
import shutil
import pathlib
import torch
import sys
import numpy as np
from torchsummary import summary
from matplotlib import pyplot as plt
from utils import *

In [3]:
# load config
cfg = None
valid_flag = True

with open('./config_base.yaml', 'r') as y_file:
    cfg = yaml.load(y_file, Loader=yaml.FullLoader)

# load model

model = get_model(cfg,model_name,device)
model = model.to(device)

if cfg["TRAIN"]["SUMMARY"]:
    print(summary(
        model,
        input_size=(
            cfg["DATASET"]["IN_CHANNELS"],
            cfg["DATASET"]["IMAGE_SIZE"] // cfg["DATASET"]["PREPROCESSING"]['DOWNSCALE_FACTOR'],
            cfg["DATASET"]["IMAGE_SIZE"] // cfg["DATASET"]["PREPROCESSING"]['DOWNSCALE_FACTOR'],
        ),
    ))

if cfg["TRAIN"]["PATH_TO_WEIGHTS"] != 'None':
    load_path = os.path.join(cfg["TRAIN"]["PATH_TO_WEIGHTS"],'{}_{}.pth'.format(model_name.lower(),SR_type))
    load_network(model, load_path, strict=True)
    
# save dir
save_path = os.path.join(cfg["TRAIN"]["PATH_TO_SAVE"],'{}_{}'.format(model_name.lower(),SR_type))
os.makedirs(save_path,exist_ok=True)


im_path,p_lr,p_hr = get_path_param(SR_type,cfg)
train_data, valid_data = load_train(cfg, p_lr)

print('{} network loaded'.format(model_name))
print('{} train image(s) loaded'.format(len(train_data)))
print('{} valid image(s) loaded'.format(len(valid_data)))

if len(valid_data) == 0:
    #For dummy training case
    valid_flag = False

# Define the loss
f_loss = get_loss(cfg).to(device)

# Define the optimizer
optimizer = get_optimizer(cfg, model.parameters())

# Define Scheduler
scheduler = get_scheduler(cfg, optimizer)

# Init Checkpoint class
checkpoint = ModelCheckpoint(
    save_path, model, cfg["TRAIN"]["EPOCH"], cfg["TRAIN"]["CHECKPOINT_STEP"]
    )

/home/cfren/anaconda3/envs/recherche/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SWINTRANSFORMER network loaded
1 train image(s) loaded
0 valid image(s) loaded


/home/cfren/anaconda3/envs/recherche/lib/python3.11/site-packages/torch/utils/data/dataset.py:414: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


In [4]:
# Start training loop
for epoch in range(cfg["TRAIN"]["EPOCH"]):
    
    # Train
    training_loss = train_one_epoch(
        model,
        train_data,
        f_loss,
        optimizer,
        device,
    )

    # Validation
    if valid_flag:
        (valid_loss, psnr, _, _, _, l1_loss, l2_loss, ssim_loss, huber_loss,) = valid_one_epoch(
            model,
            valid_loader,
            f_loss,
            device,
            cfg["DATASET"]["CLIP"]["MAX"] - cfg["DATASET"]["CLIP"]["MIN"],
        )
        print(f"EPOCH : {epoch}, Train loss: {training_loss}, Valid loss: {valid_loss}")
        print(f"PSNR: {psnr}, L1: {l1_loss}, L2: {l2_loss}, SSIM: {ssim_loss}, HUBER loss: {huber_loss}")
    else:
        #Dummy validation when no valid set
        (valid_loss, psnr, _, _, _, l1_loss, l2_loss, ssim_loss, huber_loss,) = valid_one_epoch(
            model,
            train_data,
            f_loss,
            device,
        )
        print(f"EPOCH : {epoch}, Train loss: {training_loss}, Valid loss: {valid_loss}")
        print(f"PSNR: {psnr}, L1: {l1_loss}, L2: {l2_loss}, SSIM: {ssim_loss}, HUBER loss: {huber_loss}")
        # Update scheduler
    if scheduler is not None:
        scheduler.step(valid_loss)

    # Save best model
    checkpoint.update(valid_loss, epoch)

    # Get current learning rate
    learning_rate = optimizer.param_groups[0]["lr"]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  6.23it/s]


EPOCH : 0, Train loss: 0.07482893764972687, Valid loss: 0.035746168345212936
PSNR: 59.619692602828934, L1: 0.20908474922180176, L2: 0.0715336948633194, SSIM: 0.6776777505874634, HUBER loss: 0.035746168345212936
Saving a better model


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  4.77it/s]


EPOCH : 1, Train loss: 0.035469163209199905, Valid loss: 0.01827661134302616
PSNR: 62.531589796633476, L1: 0.14518696069717407, L2: 0.03658650815486908, SSIM: 0.6613328456878662, HUBER loss: 0.01827661134302616
Saving a better model


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  5.02it/s]


...

EPOCH : 7, Train loss: 0.013055453076958656, Valid loss: 0.01297856867313385
PSNR: 64.01332748388941, L1: 0.11699434369802475, L2: 0.026010408997535706, SSIM: 0.6782691478729248, HUBER loss: 0.01297856867313385
Saving a better model


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  5.22it/s]


EPOCH : 8, Train loss: 0.012936566025018692, Valid loss: 0.01305672898888588
PSNR: 63.9893511404087, L1: 0.11968369781970978, L2: 0.026154402643442154, SSIM: 0.679003119468689, HUBER loss: 0.01305672898888588


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  5.34it/s]


EPOCH : 9, Train loss: 0.013091996312141418, Valid loss: 0.012797551229596138
PSNR: 64.07801877056299, L1: 0.11972814798355103, L2: 0.025625836104154587, SSIM: 0.684995174407959, HUBER loss: 0.012797551229596138
Saving a better model
Saving model at Epoch 9
